## Pandas analysis

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a couple of FPGAs (---> Field Programmable Gate Arrays). Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [ ]:
# If you didn't download it yet, please get the relevant file now!
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ~/data/

1\. Create a Pandas DataFrame reading N rows of the 'data_000637.txt' dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k.

In [7]:
import pandas as pd
import numpy as np
path=r'\\wsl.localhost\Ubuntu\home\user\labcomput\lab5\data\data_000637.txt'
df=pd.read_csv(path)
sel_df=df[:10001]

1310720


In [10]:
print("total original length:", len(df), '\n')
print(df[:20])

# ASK FOR GENERAL MEANING?

total original length: 1310720 

    HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS
0      1     0          123  3869200167        2374        26
1      1     0          124  3869200167        2374        27
2      1     0           63  3869200167        2553        28
3      1     0           64  3869200167        2558        19
4      1     0           64  3869200167        2760        25
5      1     0           63  3869200167        2762         4
6      1     0           61  3869200167        2772        14
7      1     0          139  3869200167        2776         0
8      1     0           62  3869200167        2774        21
9      1     0           60  3869200167        2788         7
10     1     1            7  3869200167        2785         4
11     1     0           64  3869200167        2786        19
12     1     1            6  3869200167        2792        18
13     1     0           36  3869200167        2791        23
14     1     0           56  38692001

2\. Find out the number of BX in a ORBIT (the value 'x').

In [ ]:
# ie find for how many rows BX_COUNTER has the same value (before changing); see AI for the simplest solution

3\. Find out how much the data taking lasted. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading out the whole dataset.

In [ ]:
# number of TDC_MEASs * 30ns, (group them? ie not in chronologiacl order yet?)

4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information).

In [ ]:
#sel_df.apply(30*TDC + 25*BX_COUNTER + x * BX_COUNTER)

5\. Replace the values (all 1) of the HEAD column randomly with 0 or 1.

In [ ]:
#sel_df.loc[:,'HEAD'] = np.random.Generator.integers(low=0,high=1, size = N)

6\. Create a new DataFrame that contains only the rows with HEAD=1.

In [ ]:
#new_sel_df=sel_df[sel_df['HEAD'] > 0]

7\. Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel

In [ ]:
# sel_df[sel_df['FPGA'>0, somecolumn]].plot()
# sel_df[sel_df['FPGA'<0, somecolumn]].plot()

8\. Use the groupby method to find out the noisy channels, i.e. the TDC channels with most counts (say the top 3)

In [ ]:
# sel.df.groupby(somecolumn).max, or better order them partially; maybe through apply(); ask AI for cleanest solution / read tutorial on grouping?

9\. Count the number of unique orbits. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139

In [ ]:
# wdym "unique orbits"? anw apply a filter as in point7